# コード片をメモしていく

In [1]:
import pandas as pd
import numpy as np
import re
from collections import Counter
import pyodbc
import json
from collections import OrderedDict
import datetime as dt

### アカウント情報などをテキストファイルから読み込む
yoshida_naohiro  
password  

みたいな感じのファイル

In [31]:
# ファイル読み込み
f = open("./account.txt")
lines = f.readlines()
f.close()
# 改行の除去
uid = lines[0].replace("\n","").replace("\r","")
pwd = lines[1].replace("\n", "").replace("\r","")

In [32]:
print(uid)
print(pwd)

yoshida_naohiro
password


In [34]:
# withを使う
with open("./account.txt") as f:
    lines = f.readlines()
    uid = lines[0].replace("\n","").replace("\r","")
    pwd = lines[1].replace("\n", "").replace("\r","")

print(uid)
print(pwd)

yoshida_naohiro
password


### Redshiftの接続サンプル

#### sql
```
sql = """select * from table limit 10"""
```

#### 接続設定
```
conn = pyodbc.connect(
r'DRIVER={{Amazon Redshift (x64)}};'
r'SERVER=xxxxxxxxxxxxxxxxxxxxx.redshift.amazonaws.com;'
r'Port=9999;'
r'DATABASE=database;'
r'UID={0};PWD={1};'.format(uid,pwd)
)
```

#### sqlの結果をDataFrameに格納
```
df = pd.io.sql.read_sql(sql, conn)
```

### twitterのハッシュタグ抽出

In [21]:
# サンプルdataframeの作成
# とりあえずnumpyで作る
data = np.array([
    [111111,u"あいうえお。#ハッシュタグ1 #ハッシュタグ2"]
    ,[222222, u"かきくけこ　#ハッシュタグ2　さしすせそ　#ハッシュタグ４"]
])
data

array([['111111', 'あいうえお。#ハッシュタグ1 #ハッシュタグ2'],
       ['222222', 'かきくけこ\u3000#ハッシュタグ2\u3000さしすせそ\u3000#ハッシュタグ４']],
      dtype='<U29')

In [22]:
df = pd.DataFrame(data, columns=['user_id', 'tweet'])
df

,user_id,tweet
0,111111,あいうえお。#ハッシュタグ1 #ハッシュタグ2
1,222222,かきくけこ #ハッシュタグ2 さしすせそ #ハッシュタグ４


In [23]:
# 格納用の空リスト
hashtag_list = []

# ハッシュタグの抽出
for index, row in df.iterrows():
    tmp = row.tweet.split("#")
    del tmp[0] # 1つ目の#より前の要素を削除
    hashtag_list = hashtag_list + tmp

In [24]:
hashtag_list

['ハッシュタグ1 ', 'ハッシュタグ2', 'ハッシュタグ2\u3000さしすせそ\u3000', 'ハッシュタグ４']

In [25]:
# 格納用の空リスト
hashtag_list_fix = []

# 正規表現パターン（空白文字）
pattern = re.compile(r"\s+")

# ハッシュタグ後のスペース以前のみ残す
for i in range(len(hashtag_list)):
    tmp = re.split(pattern, hashtag_list[i])[0]
    hashtag_list_fix.append(tmp)

In [26]:
hashtag_list_fix

['ハッシュタグ1', 'ハッシュタグ2', 'ハッシュタグ2', 'ハッシュタグ４']

In [27]:
# 各要素の数をカウント
cnt = Counter(hashtag_list_fix)
cnt_list = cnt.most_common()

In [28]:
cnt_list

[('ハッシュタグ2', 2), ('ハッシュタグ1', 1), ('ハッシュタグ４', 1)]

In [36]:
# 不要な文字列を削除してテキストファイルへ出力
with open("hashtag_count_result.txt", "w", encoding="utf-8", newline=None) as f:
    f.write("hashtag, cnt\n")
    for x in cnt_list:
        x = str(x)
        x = x.replace("(", "")
        x = x.replace("'", "")
        x = x.replace(")", "")
        f.write(x + "\n")

### jsonを処理する
jsonはpythonのdictionaryとして扱うことが可能

In [ ]:
# サンプル用のjsonファイル作成
sample = {
    "honoka": {
        "BWH": [
            78,
            58,
            82
        ],
        "height": 157
    },
    "eri": {
        "BWH": [
            88,
            60,
            84
        ],
        "height": 162
    },
    "kotori": {
        "BWH": [
            80,
            58,
            80
        ],
        "height": 159
    },
    "umi": {
        "BWH": [
            76,
            58,
            80
        ],
        "height": 159
    },
    "rin": {
        "BWH": [
            75,
            59,
            80
        ],
        "height": 155
    },
    "maki": {
        "BWH": [
            78,
            56,
            83
        ],
        "height": 161
    },
    "nozomi": {
        "BWH": [
            90,
            60,
            82
        ],
        "height": 159
    },
    "hanayo": {
        "BWH": [
            82,
            60,
            83
        ],
        "height": 156
    },
    "niko": {
        "BWH": [
            74,
            57,
            79
        ],
        "height": 154
    }
}

In [6]:
sample

{'eri': {'BWH': [88, 60, 84], 'height': 162},
 'hanayo': {'BWH': [82, 60, 83], 'height': 156},
 'honoka': {'BWH': [78, 58, 82], 'height': 157},
 'kotori': {'BWH': [80, 58, 80], 'height': 159},
 'maki': {'BWH': [78, 56, 83], 'height': 161},
 'niko': {'BWH': [74, 57, 79], 'height': 154},
 'nozomi': {'BWH': [90, 60, 82], 'height': 159},
 'rin': {'BWH': [75, 59, 80], 'height': 155},
 'umi': {'BWH': [76, 58, 80], 'height': 159}}

In [7]:
# jsonファイルへ書き込み
# pythonの辞書は順序が保障されていないので、
# 順序が入れ替わるのを防ぐ場合は、OrderDictを使う
fw = open("sample.json", "w")
json.dump(sample, fw, indent=4)

In [8]:
# pandasで読み込む
# 入れ子がない単純なjson形式ならこれで一発でDataFrameにできる
df_json = pd.read_json("sample.json")
df_json

,eri,hanayo,honoka,kotori,maki,niko,nozomi,rin,umi
BWH,"[88, 60, 84]","[82, 60, 83]","[78, 58, 82]","[80, 58, 80]","[78, 56, 83]","[74, 57, 79]","[90, 60, 82]","[75, 59, 80]","[76, 58, 80]"
height,162,156,157,159,161,154,159,155,159


In [9]:
# jsonで読み込む
with open("sample.json", "r") as f:
    json_data = json.load(f)

# 中身の確認
#ensure_ascii=Trueだと、日本語が\u3044のような形で出力されてしまう
#indentを指定することで、改行とindentが追加されて見やすくなる
#sort_keysを指定すると、辞書の順番がキーでソートされる
print(json.dumps(json_data, indent=2, sort_keys=True, ensure_ascii=False))

{
  "eri": {
    "BWH": [
      88,
      60,
      84
    ],
    "height": 162
  },
  "hanayo": {
    "BWH": [
      82,
      60,
      83
    ],
    "height": 156
  },
  "honoka": {
    "BWH": [
      78,
      58,
      82
    ],
    "height": 157
  },
  "kotori": {
    "BWH": [
      80,
      58,
      80
    ],
    "height": 159
  },
  "maki": {
    "BWH": [
      78,
      56,
      83
    ],
    "height": 161
  },
  "niko": {
    "BWH": [
      74,
      57,
      79
    ],
    "height": 154
  },
  "nozomi": {
    "BWH": [
      90,
      60,
      82
    ],
    "height": 159
  },
  "rin": {
    "BWH": [
      75,
      59,
      80
    ],
    "height": 155
  },
  "umi": {
    "BWH": [
      76,
      58,
      80
    ],
    "height": 159
  }
}


In [19]:
# jsonデータの操作
# dictionaryの操作と同じ

# keyの一覧を取得
print(json_data.keys())
# valueの一覧
print(json_data.values())
# key, valueの組み合わせの一覧
print(json_data.items())
print("*****")
# キーを指定して取得する
print(json_data["honoka"].keys())
print(json_data["honoka"].values())
print(json_data["honoka"]["height"])
print(json_data["honoka"]["BWH"])
print(json_data["honoka"]["BWH"][0]) # 最後はリストなので

dict_keys(['honoka', 'eri', 'kotori', 'umi', 'rin', 'maki', 'nozomi', 'hanayo', 'niko'])
dict_values([{'BWH': [78, 58, 82], 'height': 157}, {'BWH': [88, 60, 84], 'height': 162}, {'BWH': [80, 58, 80], 'height': 159}, {'BWH': [76, 58, 80], 'height': 159}, {'BWH': [75, 59, 80], 'height': 155}, {'BWH': [78, 56, 83], 'height': 161}, {'BWH': [90, 60, 82], 'height': 159}, {'BWH': [82, 60, 83], 'height': 156}, {'BWH': [74, 57, 79], 'height': 154}])
dict_items([('honoka', {'BWH': [78, 58, 82], 'height': 157}), ('eri', {'BWH': [88, 60, 84], 'height': 162}), ('kotori', {'BWH': [80, 58, 80], 'height': 159}), ('umi', {'BWH': [76, 58, 80], 'height': 159}), ('rin', {'BWH': [75, 59, 80], 'height': 155}), ('maki', {'BWH': [78, 56, 83], 'height': 161}), ('nozomi', {'BWH': [90, 60, 82], 'height': 159}), ('hanayo', {'BWH': [82, 60, 83], 'height': 156}), ('niko', {'BWH': [74, 57, 79], 'height': 154})])
*****
dict_keys(['BWH', 'height'])
dict_values([[78, 58, 82], 157])
157
[78, 58, 82]
78


### 重複除外
集合を使うことでできる

In [22]:
# サンプルのリスト
list_dup = [1111, 2222, 2222, 3333, 1111]
list_unique = list(set(list_dup))
print(list_unique)

# 文字列へ置換（おまけ）
# 文字列 = "区切り文字".join(リスト) で文字列に連結できる
# mapでリスト全体にstr()を適用して、joinで繋げている
# mapはmapオブジェクトを返すので、一度listに変換しないといけない気がするけどしなくても通るのはなぜ...？
list_str = ",".join(map(str, list_unique))
print(list_str)

[3333, 2222, 1111]
3333,2222,1111


### コマンドラインパーサー
```
import argparse
parser = argparse.ArgumentParser(description='description for arguments')
parser.add_argument('args_name', type=str, nargs=1,
                     help='help for agruments')

args = parser.parse_args()
args_str = args.args_name[0]
```

parser.add_argumentについて  
第一引数：引数の名前。この名前でparser.parse_args()から引数を呼び出す  
type：引数の型。str, intなどが指定できる  
nargs：引数の数。固定値を指定すると、その数の引数がないとエラーが出る（はず）。"\*"を指定すると、可変長引数も使用できる  
help：エラー発生時に表示されるhelp

2つ以上の異なる引数を使用する場合は、それぞれをparse.add_argumentで定義する

### pandasあれこれ

In [31]:
# あるカラムに関数を適用したカラムを新しく作る
# サンプルデータ
data = np.array([
    [1,100,200,300]
    ,[2,20,50,20]
    ,[3,50,100,150]
])

df = pd.DataFrame(data, columns=["id","value1", "value2", "value3"])
print(df)

# value1を10で割った数を新しいカラムに入れる
df["value4"] = df["value1"].apply(lambda x: x/10)
# ぶっちゃけ↓でもいけるといえばいける
df["value5"] = df.value1 / 10
print(df)

   id  value1  value2  value3
0   1     100     200     300
1   2      20      50      20
2   3      50     100     150
   id  value1  value2  value3  value4  value5
0   1     100     200     300    10.0    10.0
1   2      20      50      20     2.0     2.0
2   3      50     100     150     5.0     5.0


In [33]:
# listに含まれるidの行だけ消したい
list_exc = [123, 456]
df = pd.DataFrame({'id':[123,456,789]})
df[~(df.id.isin(list_exc))]

,id
2,789


In [35]:
dt.datetime.today()

datetime.datetime(2018, 4, 9, 21, 59, 6, 759461)

### 乱数生成を使った抽選スクリプト

In [4]:
# 抽選処理の関数を定義
# 引数は抽選対象者のDataFrameと、当選者数
def lottery(df, win_num):
    #rand_num = np.random.randint(0, len(df), win_num) # 重複除外のパラメータがないのでよくない
    rand_num = np.random.choice(len(df), win_num, replace=False) # 重複除外して乱数生成
    get_prize_user_list = list(df["id"].loc[rand_num])
    return get_prize_user_list

# 与えられた引数の数だけ抽選処理をループする関数
# 引数は抽選対象のidのDataFrameと、listにした引数
# 最後に当選者のidを以降の抽選対象から外す
def lottery_loop(df, args_list):
    # 当選者全体のdictionary
    get_prize_user_dict = {}
    # 1等から順に処理
    for i in range(len(args_list)):
        get_prize_user_list = lottery(df, args_list[i])
        get_prize_user_dict[str(i+1) + "_prize"] = get_prize_user_list
        print("*******")
        print(str(i+1) + "等の当選者: " + str(get_prize_user_list))
        print("*******")
        # csvに書き込み
        write_csv(get_prize_user_list, i+1)
        # 当選者を以降の抽選対象から除外
        df = df[~(df.id.isin(get_prize_user_list))]
        df = df.reset_index(drop=True)
    return get_prize_user_dict

# 当選者をcsvファイルに書き込む関数
# 引数は当選者のリストと、当選した等
### aggregation_timeはスクリプトの実行時間（UTC）
def write_csv(get_prize_user_list, prize_number):
    today = dt.datetime.today()
    aggregation_time = today.strftime("%Y-%m-%d %H:%M:%S")
    f = open(filename_result, mode="a", encoding="utf-8", newline=None)
    for i in range(len(get_prize_user_list)):
        f.write(str(get_prize_user_list[i]) + "," + str(prize_number) + "," + aggregation_time + "\n")
    f.close()

In [5]:
filename_result = "test.csv"
df = pd.DataFrame({"id":[111,222,333,444,555,666,777,888,999]})
args_list = [1,4]
result = lottery_loop(df, args_list)

*******
1等の当選者: [888]
*******
*******
2等の当選者: [222, 777, 666, 555]
*******
